<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Graph-Convolution-Neural-Networks" data-toc-modified-id="Graph-Convolution-Neural-Networks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Graph Convolution Neural Networks</a></span><ul class="toc-item"><li><span><a href="#Clonar-repo-en-colab" data-toc-modified-id="Clonar-repo-en-colab-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Clonar repo en colab</a></span></li><li><span><a href="#Modulos-basicos" data-toc-modified-id="Modulos-basicos-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Modulos basicos</a></span></li><li><span><a href="#Modulos-locales-al-repo" data-toc-modified-id="Modulos-locales-al-repo-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Modulos locales al repo</a></span></li><li><span><a href="#Variables-importantes-del-notebook" data-toc-modified-id="Variables-importantes-del-notebook-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Variables importantes del notebook</a></span></li></ul></li><li><span><a href="#Task-and-data:-predict-solubility-(logP)-💧" data-toc-modified-id="Task-and-data:-predict-solubility-(logP)-💧-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Task and data: predict solubility (logP) 💧</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Machine-learning-and-deep-learning-in-a-nutshell" data-toc-modified-id="Machine-learning-and-deep-learning-in-a-nutshell-2.0.0.1"><span class="toc-item-num">2.0.0.1&nbsp;&nbsp;</span>Machine learning and deep learning in a nutshell</a></span></li></ul></li></ul></li><li><span><a href="#Let's-load-some-molecular-data-with-🐼s" data-toc-modified-id="Let's-load-some-molecular-data-with-🐼s-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Let's load some molecular data with 🐼s</a></span></li><li><span><a href="#Train/test-splits-and-preprocessing" data-toc-modified-id="Train/test-splits-and-preprocessing-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Train/test splits and preprocessing</a></span></li><li><span><a href="#Exploratory-data-analysis-(EDA)" data-toc-modified-id="Exploratory-data-analysis-(EDA)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Exploratory data analysis (EDA)</a></span></li></ul></li><li><span><a href="#ML" data-toc-modified-id="ML-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>ML</a></span><ul class="toc-item"><li><span><a href="#Data-design-matrix-($X$,-$Y$)" data-toc-modified-id="Data-design-matrix-($X$,-$Y$)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Data design matrix ($X$, $Y$)</a></span></li></ul></li></ul></div>

# Graph Convolution Neural Networks

Ejecutar este notebook via colab:
<a href="https://colab.research.google.com/github/riiaa19_workshop_template/blob/master/notebooks/11_Colab_ejemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clonar repo en colab

In [5]:
import os
import subprocess
import sys

GIT_NAME='GraphNeuralNets_19'
GIT_URL='https://riiaa@github.com/riiaa/{}.git'.format(GIT_NAME)
IN_COLAB = 'google.colab' in sys.modules

def run_cmd(cmd):
    print('Output of "{}":'.format(cmd))
    print(subprocess.run(cmd,stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8'))

def run_cmd_list(cmd_list):
    [run_cmd(cmd) for cmd in cmd_list]
        

def install_rdkit():
    cmd_list = [
        "wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh",
        "chmod +x Miniconda3-latest-Linux-x86_64.sh",
        "bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local",
        "conda install -q -y -c conda-forge rdkit",
        "rm -rf Miniconda3-latest-Linux-x86_64.sh"]
    print('Installing rdkit\n\n')
    run_cmd_list(cmd_list)
    custom_path = '/usr/local/lib/python3.7/site-packages/'
    print('Do not forget to append "{}"" to sys.path'.format(custom_path))

if IN_COLAB:
    SRC_DIR='.'
    run_cmd('apt install graphviz libgraphviz-dev libcgraph6')
    run_cmd('rm -rf sample_data')
    run_cmd('rm -rf {}'.format(GIT_NAME))
    run_cmd('git clone --verbose --progress {}'.format(GIT_URL))
    run_cmd('mv {}/* . '.format(GIT_NAME))
    run_cmd('rm -rf {}'.format(GIT_NAME))
    run_cmd('pip install spektral')
    install_rdkit()
else:
    SRC_DIR='..'
    
print('Using colab? {}, using root directory "{}"'.format(IN_COLAB,SRC_DIR))

Using colab? False, using root directory ".."


In [ ]:
import sys
if 'google.colab' in sys.modules:
    sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [4]:
pip install spektral

  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7fe716aab9b0>: Failed to establish a new connection: [Errno -2] Name or service not known')': /simple/spektral/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7fe716aab6a0>: Failed to establish a new connection: [Errno -2] Name or service not known')': /simple/spektral/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7fe716aab5f8>: Failed to establish a new connection: [Errno -2] Name or service not known')': /simple/spektral/
  Retrying (Retry(total=1, connect=None, read=None, redirect=None, status

## Modulos basicos

In [6]:
from collections import OrderedDict

from tqdm import tqdm

# scientific python stack
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Chemo-informatics
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw.MolDrawing import MolDrawing
from rdkit.Chem.Draw import IPythonConsole
print('RDKit:{}'.format(rdkit.__version__))
# ML/DL
import sklearn
import sklearn.model_selection
import sklearn.metrics
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

print('Tensorflow:{}'.format(tf.__version__))
print('Keras:{}'.format(keras.__version__))

/home/beangoben/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


## Modulos locales al repo

In [7]:
sys.path.append(os.path.join(SRC_DIR,'code'))
import utils

## Variables importantes del notebook

In [8]:
data_path = os.path.join(SRC_DIR,'data')

# Task and data: predict solubility (logP) 💧

Datatset source: Delaney's solubility dataset from ESOL:  Estimating Aqueous Solubility Directly from Molecular Structure
(https://pubs.acs.org/doi/10.1021/ci034243x)

 
#### Machine learning and deep learning in a nutshell

![](https://github.com/beangoben/chemistry_ml_colab/blob/master/images/ml_dl.png?raw=true)


## Let's load some molecular data with 🐼s

In [2]:
df = pd.read_csv('data/solubility.csv')
print(df.shape)
print(df.columns)
smiles_column='smiles'
target='measured log solubility in mols per litre'
df.head()

NameError: name 'pd' is not defined

## Train/test splits and preprocessing

In [1]:
y = df[target].values.reshape(-1,1)
print(y.shape)


indices=df.index.tolist()
train_index,test_index = sklearn.model_selection.train_test_split(indices)
print(len(train_index),len(test_index))

sns.distplot(y)
plt.xlabel(target)
plt.show()

NameError: name 'df' is not defined

## Exploratory data analysis (EDA)

# ML
## Data design matrix ($X$, $Y$)